# Writing Efficient Code and GPU Computing Homework

Please save your solutions as a **PDF** and upload it to Canvas.

## Problem 1: Profiling and Vectorization

**(a)** Consider the following cProfile output from a data analysis program:

```
   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     5000    8.234    0.002    8.234    0.002 analysis.py:12(compute_distances)
     5000    0.089    0.000    0.089    0.000 analysis.py:28(normalize_vector)
  2500000    1.456    0.000    1.456    0.000 analysis.py:35(squared_diff)
        1    0.002    0.002    9.781    9.781 analysis.py:50(main)
```

Which function should you optimize first? Explain your reasoning based on the profiling data. What percentage of the total runtime does this function account for?

We should optimize the compute_distances first, as which has the longest run time per call. The percentage of total time it takes 8.234/9.781 = 84.2%

**(b)** The following function computes weighted squared differences between two arrays:

In [1]:
def weighted_squared_diff_loop(x, y, w):
    """Compute sum of weighted squared differences using a loop."""
    n = len(x)
    total = 0.0
    for i in range(n):
        diff = x[i] - y[i]
        total += w[i] * diff * diff
    return total

Write a vectorized version of this function using NumPy operations. Your function should produce the same result but without explicit Python loops. For example, `weighted_squared_diff(np.array([1, 2, 3]), np.array([0, 1, 1]), np.array([1, 2, 3]))` should return `15.0`.

In [2]:
import numpy as np


def weighted_squared_diff(x, y, w):
    """Compute sum of weighted squared differences using vectorization."""
    return np.sum(w*(x-y)**2)

**(c)** Write a function that transforms an array by replacing negative values with zero and scaling all positive values by their mean. For example, given `np.array([-2, 4, -1, 6, 2])`, the positive values are `[4, 6, 2]` with mean `4.0`, so the result should be `np.array([0, 1, 0, 1.5, 0.5])`. Use boolean indexing instead of loops.

In [3]:
import numpy as np


def transform_array(arr):
    """Replace negatives with 0, scale positives by their mean."""
    result = np.zeros_like(arr, dtype=float)

    pos_mask = arr > 0
    mean_pos = arr[pos_mask].mean()

    result[pos_mask] = arr[pos_mask] / mean_pos
    return result

## Problem 2: Parallelization and JIT Compilation

**(a)** The following function computes the mean of a bootstrap sample:

In [4]:
def compute_bootstrap_mean(args):
    """Compute mean of a bootstrap sample."""
    data, seed = args
    rng = np.random.RandomState(seed)
    sample = rng.choice(data, size=len(data), replace=True)
    return np.mean(sample)

Write a function that uses `multiprocessing.Pool` to compute `n_bootstrap` bootstrap means in parallel. Each bootstrap iteration should receive a unique seed to ensure different random samples. Return a list of the bootstrap means.

In [5]:
import numpy as np
import multiprocessing as mp


def compute_bootstrap_mean(args):
    """Compute mean of a bootstrap sample."""
    data, seed = args
    rng = np.random.RandomState(seed)
    sample = rng.choice(data, size=len(data), replace=True)
    return np.mean(sample)


def parallel_bootstrap(data, n_bootstrap, n_workers=4):
    """Compute bootstrap means in parallel."""
    args_list = [(data, seed) for seed in range(n_bootstrap)]
    with mp.Pool(processes=n_workers) as pool:
        results = pool.map(compute_bootstrap_mean, args_list)
    return results

**(b)** Write a Numba-optimized function that computes the running maximum of an array. For each position `i`, the output should contain the maximum of all elements from index 0 to `i` (inclusive). For example, `running_max(np.array([3, 1, 4, 1, 5, 9, 2, 6]))` should return `np.array([3, 3, 4, 4, 5, 9, 9, 9])`.

In [6]:
from numba import njit
import numpy as np


@njit
def running_max(arr):
    """Compute running maximum of array."""
    n = len(arr)
    if n == 0:
        return arr.copy()

    result = np.empty(n, dtype=arr.dtype)

    current_max = arr[0]
    result[0] = current_max

    for i in range(1, n):
        if arr[i] > current_max:
            current_max = arr[i]
        result[i] = current_max

    return result

**(c)** The following Numba function attempts to filter an array to keep only positive values, but it fails to compile. Explain why it fails and provide a corrected version that compiles successfully with `@njit`.

In [ ]:
from numba import njit
import numpy as np


@njit
def filter_positive_broken(arr):
    """Return array containing only positive values (BROKEN)."""
    count = 0
    for x in arr:
        if x > 0:
            count += 1
    result = np.empty(count, dtype=arr.dtype)

    idx = 0
    for x in arr:
        if x > 0:
            result[idx] = x
            idx += 1

    return result

The previous code failed to compile as the it cannot infer the type of empty python list.

## Problem 3: GPU Computing Fundamentals

**(a)** For each of the following computational tasks, state whether it would benefit from GPU acceleration and explain why or why not.

1. Computing the mean of 500 numbers
No. Transferring data to and from the GPU would dominate the runtime.

2. Multiplying two 5000x5000 matrices
Yes. Matrix multiplication is highly parallelizable and computationally intensive.

3. Reading a 10GB CSV file from disk
No. This is not related to computation method.

4. Running 1 million independent Monte Carlo simulations
Yes. This could be highly parallelizable when using GPU.

5. Computing Fibonacci numbers recursively
No. Recursive Fibonacci computation has strong data dependencies and poor parallelism.

**(b)** The following code runs slowly despite using GPU. Identify the performance problem and rewrite the code to fix it. The goal is to compute the sum of squares for 1000 different arrays.

In [3]:
import cupy as cp
import numpy as np

# results = []
# for i in range(1000):
    # data = np.random.randn(10000)  # Generate on CPU
    # gpu_data = cp.asarray(data)    # Transfer to GPU
    # result = cp.sum(gpu_data ** 2) # Compute on GPU
    # results.append(result.get())   # Transfer back to CPU
##The issue is that the data been transfered too often between CPU and GPU

##reivsed code
gpu_data = cp.random.randn(1000, 10000)
results = cp.sum(gpu_data ** 2, axis=1)

# Move result to CPU only once
total = cp.sum(results).get()



print(f"Total: {total}")

Total: 10005309.023272444


Write an efficient version that minimizes data transfers between CPU and GPU.

## Problem 4: CuPy and PyTorch

**(a)** Convert the following NumPy code to CuPy. The function computes z-score normalization and then the correlation matrix.

In [ ]:
import numpy as np


def correlation_matrix_numpy(X):
    """Compute correlation matrix after z-score normalization.

    X has shape (n_samples, n_features).
    """
    # Z-score normalize each column
    mean = np.mean(X, axis=0)
    std = np.std(X, axis=0)
    Z = (X - mean) / std

    # Compute correlation matrix
    n = X.shape[0]
    corr = (Z.T @ Z) / n
    return corr

Write the CuPy version that performs the computation on GPU and returns the result as a NumPy array.

In [ ]:
import cupy as cp
import numpy as np


def correlation_matrix_cupy(X):
    """Compute correlation matrix using CuPy (GPU)."""
    # Move data to GPU
    X_gpu = cp.asarray(X)

    # Z-score normalize each column
    mean = cp.mean(X_gpu, axis=0)
    std = cp.std(X_gpu, axis=0)
    Z = (X_gpu - mean) / std

    # Compute correlation matrix
    n = X_gpu.shape[0]
    corr_gpu = (Z.T @ Z) / n

    # Move result back to CPU
    return cp.asnumpy(corr_gpu)

**(b)** The following PyTorch code has a bug that causes a runtime error. Identify the error and provide the corrected code.

weights is created on the CPU, while x is on the GPU, causing a device mismatch during multiplication. Below is the revised code.

In [17]:
import torch
import numpy as np


def process_data(numpy_array):
    """Process data using PyTorch on GPU."""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Convert to tensor and move to GPU
    x = torch.from_numpy(numpy_array).to(device)

    # Changed here
    weights = torch.ones(len(numpy_array), device=device)


    # Weighted sum
    result = torch.sum(x * weights)

    return result.item()

**(c)** Explain why the following GPU timing code gives incorrect measurements. Then provide corrected code that accurately measures GPU computation time.

The timing is incorrect because GPU operations are asynchronous and the code does not synchronize the GPU before stopping the timer.

In [25]:
import torch
import time

device = torch.device('cuda')
a = torch.randn(5000, 5000, device=device)
b = torch.randn(5000, 5000, device=device)

start = time.perf_counter()
c = torch.mm(a, b)
torch.cuda.synchronize()   # Wait for GPU to finish
elapsed = time.perf_counter() - start
print(f"Time: {elapsed*1000:.2f} ms")

Time: 88.78 ms


## Problem 5: Performance Comparison

**(a)** In extreme value statistics, we often need to estimate the probability that the maximum of n independent standard normal random variables exceeds a threshold t. This can be done via Monte Carlo simulation: generate n normal values, take the maximum, and check if it exceeds t. Repeat this many times and compute the proportion that exceed t.

Implement two versions of this simulation:

1. A Numba-optimized CPU version using `@njit`
2. A CuPy GPU version using vectorized operations

Both functions should take parameters `n` (number of normal values per trial), `t` (threshold), and `n_simulations` (number of Monte Carlo trials), and return the estimated probability.

In [26]:
from numba import njit
import numpy as np
import cupy as cp


@njit
def estimate_prob_numba(n, t, n_simulations):
    """Estimate P(max of n normals > t) using Numba."""
    count = 0

    for _ in range(n_simulations):
        # Initialize max with first sample (no hard-coded value)
        max_val = np.random.randn()

        for _ in range(1, n):
            x = np.random.randn()
            if x > max_val:
                max_val = x

        if max_val > t:
            count += 1

    return count / n_simulations


def estimate_prob_cupy(n, t, n_simulations):
    """Estimate P(max of n normals > t) using CuPy."""
    # Generate all samples on GPU
    samples = cp.random.randn(n_simulations, n)

    # Max per simulation
    max_vals = cp.max(samples, axis=1)

    # Compute probability
    prob = cp.mean(max_vals > t)

    # Return scalar to CPU
    return float(prob.get())

**(b)** Design an experiment to find the "crossover point" where the GPU version becomes faster than the CPU version. Your experiment should vary the problem size (e.g., `n_simulations`) and measure execution time for both implementations. Describe what factors affect where this crossover occurs and what values you would test.

Fix n and t, and vary n_simulations over a wide range (e.g., 10^3 to 10^8). For each value, measure the runtime of the Numba CPU version and the CuPy GPU version. The crossover point is the smallest n_simulations at which the GPU runtime becomes consistently lower than the CPU runtime, reflecting when GPU parallelism outweighs its overhead.



**(c)** Suppose you need to run a very large simulation with `n_simulations = 100_000_000` but your GPU only has 8GB of memory. The naive CuPy implementation would require generating a matrix of shape `(n_simulations, n)` which may not fit in memory. Write a batched version that processes the simulations in chunks to stay within memory limits.

In [ ]:
import cupy as cp


def estimate_prob_cupy_batched(n, t, n_simulations, batch_size): ##need to clear the batch size
    """Estimate P(max of n normals > t) using CuPy with batching."""
    exceed_count = 0
    remaining = n_simulations

    while remaining > 0:
        current_batch = min(batch_size, remaining)

        # Generate batch on GPU
        samples = cp.random.randn(current_batch, n)

        # Compute max per simulation
        max_vals = cp.max(samples, axis=1)

        # Count exceedances
        exceed_count += int(cp.sum(max_vals > t).get())

        remaining -= current_batch

    return exceed_count / n_simulations